In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import re
import scipy.optimize as opt
from tqdm import tqdm
import sympy as sp
from sympy.parsing.sympy_parser import parse_expr
from sympy import lambdify

In [3]:
#filename = 'pareto_low_adv.csv'
filename = 'pareto_low_sim.csv'

In [4]:
t_eq=pd.read_csv('/data/zj448/SR/Ultimate_paper/pareto_archive/'+filename)
len(t_eq)

280815

In [5]:
# drop rows that have the same value in column sympy_format.
t_eq=t_eq.drop_duplicates(subset='sympy_format',keep='first')
len(t_eq)

627

In [6]:
t_eq['fitting_format']=''
t_eq['num_fitting_variables']=0
t_eq['initial_constant_guess'] = pd.Series(dtype='object')

number_matching_pattern = r"(?<![a-zA-Z0-9_.])[+-]?(\d+\.\d+|\.\d+|\d+\.|\d+)(?:[eE][-+]?\d+)?"

for row in tqdm(t_eq.iterrows(),total=len(t_eq)):
    #equation = row[1]['sympy_format']
    #equation = parse_expr(equation)

    raw_equation = row[1]['equation']
    # replace the power sign
    raw_equation = raw_equation.replace('^','**')
    equation = str(parse_expr(raw_equation))

    # # swap constants
    # constants = re.findall(number_matching_pattern, equation)


    # variable_list=[]
    # for i in range(len(constants)):
    #     variable_list.append('p['+str(i)+']')


    # for i in range(len(constants)):
    #     equation = equation.replace(constants[i],variable_list[i])

    # swap constants v2
    # Find all unique matches first to avoid duplicates and incorrect indexing
    constants = list(set(re.findall(number_matching_pattern, equation)))
    # Sort constants by their length in descending order to replace longer numbers first, preventing partial replacement issues
    constants.sort(key=len, reverse=True)

    def replace_with_variable(match):
        # Find the matched number in the constants list and get its index
        number = match.group(1)
        index = constants.index(number)

        # Check if the match includes a minus sign
        if match.group(0).startswith('-'):
            sign = '-'
        else:
            sign = ''
        return f'{sign}p[{index}]'

    equation = re.sub(number_matching_pattern, replace_with_variable, equation)

    t_eq.loc[row[0],'fitting_format']=str(equation) 
    t_eq.loc[row[0],'num_fitting_variables']=len(constants)
    t_eq.at[row[0],'initial_constant_guess']=constants

  0%|          | 0/627 [00:00<?, ?it/s]

100%|██████████| 627/627 [00:02<00:00, 220.16it/s]


In [7]:
t_eq=t_eq.drop_duplicates(subset='fitting_format',keep='last')
len(t_eq)

463

In [8]:
#t_eq.to_csv('/data/zj448/SR/Ultimate_paper/pareto_archive/pareto_cleaned.csv',index=False)
t_eq.to_csv('/data/zj448/SR/Ultimate_paper/pareto_archive/'+filename[:-4]+'_cleaned.csv',index=False)

In [9]:
t_eq

,complexity,loss,score,equation,sympy_format,lambda_format,number_constants,variables,number_variables,unique_number_variables,evolutions,iterations,fitting_format,num_fitting_variables,initial_constant_guess
0,1,4.859613,0.000000,x61,x61,PySRFunction(X=>x61),0,{'x61'},1,1,0,0,x61,0,[]
4,11,0.112913,0.043123,(x12 - ((2.361378533548963 - ((x61 / x45) - 0....,x12 - x73 - 2.483870364573009 + x61/x45,PySRFunction(X=>x12 - x73 - 2.483870364573009 ...,2,"{'x45', 'x61', 'x73', 'x12'}",4,4,0,0,x12 - x73 - p[0] + x61/x45,1,[2.483870364573009]
5,13,0.110034,0.012911,(x12 - (((2.361378533548963 - (x61 / (x45 - -0...,x12 + x61/(x45 + 0.7819512493710871) - x73 - 2...,PySRFunction(X=>x12 + x61/(x45 + 0.78195124937...,3,"{'x45', 'x61', 'x73', 'x12'}",4,4,0,0,x12 + x61/(x45 + p[0]) - x73 - p[1],2,"[0.7819512493710871, 2.43920200603072]"
6,15,0.107942,0.009601,(x12 - ((2.361378533548963 - ((x61 / ((x42 + x...,x12 + x42*x61/(x42 + x45) - x73 - 2.4838703645...,PySRFunction(X=>x12 + x42*x61/(x42 + x45) - x7...,2,"{'x45', 'x73', 'x42', 'x61', 'x12'}",6,5,0,0,x12 + x42*x61/(x42 + x45) - x73 - p[0],1,[2.483870364573009]
7,17,0.098634,0.045086,(x12 - (2.361378533548963 - (((0.4883772896702...,x12 - 2.5585015935188197 + 0.75912583848231723...,PySRFunction(X=>x12 - 2.5585015935188197 + 0.7...,5,"{'x42', 'x26', 'x12'}",4,3,0,0,x12 - p[1] + p[0]/(-p[2]*x26 + x42**p[3]),4,"[0.75912583848231723, 2.5585015935188197, 1.27..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271227,9,0.028624,0.232517,((x15 * 3.4911837061982536) - ((x42 - x39) / x...,3.4911837061982536*x15 - (-x39 + x42)/x15,PySRFunction(X=>3.4911837061982536*x15 - (-x39...,1,"{'x39', 'x42', 'x15'}",4,3,4,4128,p[0]*x15 - (-x39 + x42)/x15,1,[3.4911837061982536]
271239,11,0.023165,0.105798,((x15 * 3.4911837061982536) - ((x42 - x39) / (...,3.4911837061982536*x15 - (-x39 + x42)/(-x4 + x51),PySRFunction(X=>3.4911837061982536*x15 - (-x39...,1,"{'x51', 'x15', 'x4', 'x42', 'x39'}",5,5,4,4129,p[0]*x15 - (-x39 + x42)/(-x4 + x51),1,[3.4911837061982536]
277958,7,0.041813,0.245612,((x15 * 3.9938613107087564) - (x42 / x39)),3.9938613107087564*x15 - x42/x39,PySRFunction(X=>3.9938613107087564*x15 - x42/x39),1,"{'x39', 'x42', 'x15'}",3,3,4,4740,p[0]*x15 - x42/x39,1,[3.9938613107087564]
278081,11,0.022001,0.131577,((x15 * 3.4911837061982536) - ((x42 - x39) / (...,3.4911837061982536*x15 - 1.7676790306349228*(-...,PySRFunction(X=>3.4911837061982536*x15 - 1.767...,2,"{'x39', 'x42', 'x15'}",4,3,4,4751,p[0]*x15 - p[1]*(-x39 + x42)/x39,2,"[3.4911837061982536, 1.7676790306349228]"


In [10]:
t_eq.iloc[5]

complexity                                                                21
loss                                                                0.076269
score                                                               0.064288
equation                   (((x61 / 1.1277639790006166) + (x0 * x71)) + (...
sympy_format               x0*x71 - 0.4151472548160585*x0 - 0.41514725481...
lambda_format              PySRFunction(X=>x0*x71 - 0.4151472548160585*x0...
number_constants                                                           4
variables                          {'x73', 'x42', 'x61', 'x53', 'x71', 'x0'}
number_variables                                                           7
unique_number_variables                                                    6
evolutions                                                                 0
iterations                                                                 0
fitting_format             x0*x71 - p[2]*x0 - p[2]*x42 + p[2]*x53 + p[0]*...